[A great lecture](https://research.computing.yale.edu/training/parallel-programming-python)

## Serial Execution
Typical programs operate lines sequentially

In [1]:
import numpy as np
import os
import random

In [2]:
# define an array of numbers

foo = np.array([0,1,2,3,4,5])

# define a func tht squares the numbers
def bar(x):
    return x*x

# a loop over each element and perfrom an action on it
for element in foo:
    #print the results of bar
    print(bar(element))

0
1
4
9
16
25


# The `map` function
A key tool we will utilize later is cal `map`. This lets us apply a function to each element in an array or list


In [3]:
# verity the ineffcient way to define a map function
def my_map(function, array):
    #create a containeer for the results
    output = []
    # loop over each element
    for element in array:
        # add the intermediate result to the container
        output.append(function(element))
        
    # return the now-filled contianer
    return output

In [4]:
my_map(bar, foo)

[0, 1, 4, 9, 16, 25]

Python has a helpful `map` function in the standard libaray
this map is much more flexible and featured than ours, so it is best to use this instead


In [5]:
list(map(bar,foo))

[0, 1, 4, 9, 16, 25]

## Parallel workes
In the example we showed before, no step of the `map` call depend on the other steps  
Rather than waiting for the function to loop over each value, we could create mutliple instances of the function `bar` and apply it to each value simultaneously  
This is achieved with the `mutliprocessing` module and a pool of workers...

# The `multiprocessing` Module
The `multiprocessing` module has a number of functions to help simplify parallel processing. 
One such tool is `Pool` class. It allows us to setup  group of processes to execute tasks in parallel. This is called a pool of workeer processes.
First we will create the pool with a sepcified number of workeers. We will then use our `map` utility to apply functions to our array

In [6]:
import multiprocessing 

# creating  a pool of processes
with multiprocessing.Pool(processes=6) as pool:
    # map the np.sqaure funtion on our foo arary
    result = pool.map(np.square, foo)
    
print(result)

[0, 1, 4, 9, 16, 25]


the difference here is that each element of this this list is being handled by a different process

In [7]:
# to show how this is actually being handled, let's create a new function:

def parallel_test(x):
    # print the index of the job and it's process ID number
    print(f"x = {x}, ID = {os.getpid()}")

In [8]:
# Now we can map this function on the foo aray from before, first with simple  build-in map func
list(map(parallel_test, foo))

x = 0, ID = 130208
x = 1, ID = 130208
x = 2, ID = 130208
x = 3, ID = 130208
x = 4, ID = 130208
x = 5, ID = 130208


[None, None, None, None, None, None]

In [9]:
# now we can run this using multiprocessing; so the PID will be difference (a different process), the order will be also different
with multiprocessing.Pool(processes=6) as pool:
    result = pool.map(parallel_test, foo)

x = 0, ID = 130239x = 4, ID = 130243x = 3, ID = 130242x = 1, ID = 130240x = 2, ID = 130241x = 5, ID = 130244







**Now we tried the same process using multiprocessing:**

Two things are worth noting:
1. Each element is processed by a different PID
2. The tasks are not executed in order


# Key Take-aways:
1. The `map` fuction is designed to apply thee same functin to each of item in an iterator
2. In serial processing, this works like a for-loop
3. Parallel execution sets up multiple worker processes that act separatly and simlultaneously

# Classes of Parallelism
## Embarassingly Parallel Probelms:
- Many problems can be simply converted into parallel execution with the `multiprocessing` module

### Example 1 Monte Carlo Pi Calculation
- Run multiple instances of the same simulatioon with different random number generator seed
- Define a function to calculate `pi` that takes the random seed as input, then map it to an array of random seeds.


In [10]:
def pi_mc(seed):
    num_trails = 100000
    counter = 0
    for j in range(num_trails):
        x_val=random.random()
        y_val=random.random()
        
        radius = x_val**2 + y_val**2
        if radius <1:
            counter+=1
    return 4*counter/num_trails



## Serial vs Parallel

In [11]:
%timeit pi_mc(1)

34.4 ms ± 315 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
seed_arry=list(range(4))

%timeit list(map(pi_mc, seed_arry))


140 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%time 
with multiprocessing.Pool(processes=4) as pool:
    result = pool.map(pi_mc, seed_arry)
print(result)

[3.14368, 3.14712, 3.14488, 3.14076]
CPU times: user 11.3 ms, sys: 8.86 ms, total: 20.1 ms
Wall time: 87.8 ms


---

### Example 2: Processing multiple input files